# Output Parsers

Output parsers in Langchain are specialized classes that transform the output of language models (LLMs) into a more suitable format. This is useful when using LLMs to generate structured data.

→<i>(Trình phân tích cú pháp đầu ra trong Langchain là các lớp chuyên biệt giúp chuyển đổi đầu ra của mô hình ngôn ngữ (LLM) sang định dạng phù hợp hơn. Điều này hữu ích khi sử dụng LLM để tạo dữ liệu có cấu trúc.)</i>

There are two main methods an output parser must implement:
(Có hai phương thức chính mà output parser phải triển khai:)

- <b>Get format instructions</b>: a method that returns a string containing instructions for how the output of a language model should be formatted (một method trả về một chuỗi chứa hướng dẫn về cách định dạng đầu ra của mô hình ngôn ngữ).
- <b>Parse</b>: a method that takes in the string (assumed to be the response from a language model) and parses it into some structure. (một method lấy chuỗi (được coi là phản hồi từ mô hình ngôn ngữ) và phân tích chuỗi đó thành một cấu trúc nào đó.)

![output_parser](output-parser.png)

In [8]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [9]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [10]:
model = ChatOpenAI(temperature=0)

## CSV Parser

This output parser can be used to return a list of comma-separated items.

<i>(Trình phân tích cú pháp đầu ra này có thể được sử dụng khi bạn muốn trả về danh sách các mục được phân tách bằng dấu phẩy.)</i>

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five places {places}.\n{format_instructions}",
    input_variables=["places"],
    partial_variables={"format_instructions":format_instructions}
)
prompt

PromptTemplate(input_variables=['places'], partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz`'}, template='List five places {places}.\n{format_instructions}')

In [12]:
chain = prompt | model | output_parser
response = chain.invoke({"places":"for c summer vacation in Italy"})
response

['1. Amalfi Coast\n2. Tuscany\n3. Sicily\n4. Lake Como\n5. Cinque Terre']

## JSON Parser

The output parser allows users to specify a JSON schema and query LLMs for output that conforms to that schema. Keep in mind that large language models are leaky abstractions! You must use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, Davinci can do reliably, but Curie's ability dropped off dramatically.

<i>(Trình phân tích cú pháp đầu ra cho phép người dùng chỉ định lược đồ JSON và truy vấn LLM để tìm đầu ra phù hợp với lược đồ đó. Hãy nhớ rằng các mô hình ngôn ngữ lớn là sự trừu tượng bị rò rỉ! Bạn phải sử dụng LLM có đủ dung lượng để tạo JSON đúng định dạng. Trong dòng OpenAI, Davinci có thể làm được điều đó một cách đáng tin cậy, nhưng khả năng của Curie đã giảm sút đáng kể.)</i>

### The following example uses Pydantic to declare your data model

In [13]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate

In [14]:
# Define your desired data structure
class Travel(BaseModel):
    place: str = Field(description="name of the places")
    description: str = Field(description="description of the place")
    activities: str = Field(description="what to do in that place")

In [16]:
# Query intended to prompt a language model to populate the data structure
travel_query = "SUggest a place in France for going on trip this summer to avoid heat."
# cset up a parser + inject instructions into the prompt template 
#(Thiết lập trình phân tích cú pháp + đưa hướng dẫn vào mẫu lời nhắc)
parser = JsonOutputParser(pydantic_object=Travel)
prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
chain = prompt | model | parser
chain.invoke({"query":travel_query})

{'place': 'Chamonix',
 'description': 'Chamonix is a beautiful town located in the French Alps, known for its stunning mountain views and outdoor activities such as hiking, skiing, and mountaineering. The cool mountain air makes it a perfect destination to escape the summer heat.',
 'activities': "In Chamonix, you can enjoy activities such as hiking in the mountains, taking a cable car ride to see the panoramic views, exploring the charming town center with its shops and cafes, and even trying your hand at glacier climbing. It's a great place to relax and enjoy nature while staying cool in the summer."}

### Without Pydantic 

In [17]:
travel_query = "Suggest a place in France for going on trip this summer to avoid heat."
parser = JsonOutputParser()
prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
chain = prompt | model | parser
chain.invoke({"query":travel_query})

{'destination': 'Chamonix',
 'reason': 'Chamonix is a great destination in France for summer as it is located in the French Alps and offers cooler temperatures compared to other parts of the country. You can enjoy outdoor activities like hiking, mountain biking, and even skiing on the glaciers. Plus, the stunning views of Mont Blanc and the surrounding mountains make it a perfect summer getaway.'}

### Structured Output Parser

This output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, this is useful for less powerful models.

<i>(Trình phân tích cú pháp đầu ra này có thể được sử dụng khi bạn muốn trả về nhiều trường. Mặc dù trình phân tích cú pháp Pydantic/JSON mạnh hơn nhưng điều này lại hữu ích cho các mô hình kém mạnh mẽ hơn.)</i>

In [24]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
travel_query = "Suggest a place in France for going on trip this summer to avoid heat."
response_schemas = [
    ResponseSchema(name="answer",description="answer to the user's question"),
    ResponseSchema(name="description",description="detailed description on the answer topic"),
    ResponseSchema(name="applications",description="real-world applications of the answer topic") # ứng dụng thực tế của chủ đề câu trả lời
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template = "Answer the user question as the best as posible.\n{format_instructions}\n{question}\n",
    input_variables=["question"],
    partial_variables={"format_instructions":format_instructions}
)
chain = prompt | model | output_parser
chain.invoke({"question":travel_query})

{'answer': 'Chamonix-Mont-Blanc',
 'description': 'Chamonix-Mont-Blanc is a picturesque town located in the French Alps, known for its stunning mountain views and cooler temperatures even in the summer months. It is a popular destination for outdoor activities such as hiking, climbing, and mountain biking.',
 'applications': 'Travelers looking to escape the heat can enjoy the refreshing mountain air and explore the beautiful landscapes of Chamonix-Mont-Blanc. The town also offers charming cafes, shops, and cultural attractions for visitors to enjoy.'}